# Bayesian Statistics

In Section 5, you will learn about Bayesian statistics through looking at examples from rare disease diagnosis and baseball.

After completing Section 5, you will be able to:

- Apply Bayes' theorem to calculate the probability of A given B.
- Understand how to use hierarchical models to make better predictions by considering multiple levels of variability.
- Compute a posterior probability using an empirical Bayesian approach.
- Calculate a 95% credible interval from a posterior probability.

Some key points to consider:

- In the urn model, it does not make sense to talk about the probability of $p$ being greater than a certain value because $p$ is a fixed value.
- With Bayesian statistics, we assume that $p$ is in fact random, which allows us to calculate probabilities related to $p$.
- Hierarchical models describe variability at different levels and incorporate all these levels into a model for estimating $p$.

## Bayes' Theorem

### Theorem

Let $P$ be a probability meassure over $X$. Let $A \subseteq X$ and $B \subseteq X$ be two events such that $P(B) \neq 0$. Then the following equation holds:
$$
    P(A | B) = \frac{P(B|A)P(A)}{P(B)}.
$$

#### Proof

By definition consider the following two equations:
$$
\begin{align}
    P(A | B) & = \frac{P(A \cap B)}{P(B)} \text{, if } P(B) \neq 0,\\
    P(B | A) & = \frac{P(A \cap B)}{P(A)} \text{, if } P(A) \neq 0.
\end{align}
$$
Solving (1) and (2) for $P(A \cap B)$ and substituting we have:
$$
    P(B) P(A | B) = P(A) P(B | A) \\
    \Rightarrow P(A|B) = \frac{P(B|A) P(A)}{P(B)}
$$

Some key points to consider:

- Bayes' Theorem states that the probability of event A happening given event B is equal to the probability of both A and B divided by the probability of event B:
$$
    \mathbb{P}(A|B) = \frac{\mathbb{P}(B|A) \mathbb{A}}{\mathbb{P}(B)}
$$
- Bayes' Theorem shows that a test for a very rare disease will have a high percentage of false positives even if the accuracy of the test is high.

### Equations: Cystic fibrosis test probabilities

In these probabilities, $+$ represents a positive test, $-$ represents a negative test, $D = 0$ indicates no disease, and $D = 1$ indicates the disease is present.

Probability of having the disease given a positive test: $\mathbb{P}(D = 1 | +)$
99% test accuracy when disease is present: $\mathbb{P}(+ | D = 1) = 0.99$
99% test accuracy when disease is absent: $\mathbb{P}(- | D = 0) = 0.99$
Rate of cystic fibrosis: $\mathbb{P} = 0.00025$
Bayes' theorem can be applied like this:
$$
\begin{align*}
    \mathbb{P}(D = 1 | +) & = \frac{\mathbb{P}(+ | D=1) \cdot \mathbb{P}(D = 1)}{\mathbb{P}(+)} \\
    & = \frac{\mathbb{P}(+ | D = 1) \cdot \mathbb{P}(D = 1)}{\mathbb{P}(+ | D = 1) \cdot \mathbb{P}(D = 1) + \mathbb{P}(+ | D = 0) \cdot \mathbb{P}(D = 0)}
\end{align*}
$$
Substituting known values, we obtain:
$$
    \frac{0.99 \cdot 0.00025}{0.99 \cdot 0.00025 + 0.01 \cdot 0.99975} = 0.02
$$

Now we will create a Monte Carlo simulation of the previous exercise:

In [8]:
# Use StatsBase to make use of the sample with Weights.
using StatsBase
# Use DataFrames to create a table and work with it.
using DataFrames
# Use MLJ to compute the cofusion matrix.
using MLJ

# Disease prevalence
# prev = 0.00025
prev = 1 / 3900
# Number of tests.
N = 100000
signs = ["+", "-"]
status = ["Disease", "Healthy"]
statusDict = Dict(zip(signs, status))
outcome = sample(status, Weights([prev, 1 - prev]), N)

# Number with disease.
N₊ = map(oc -> oc == statusDict["+"], outcome) |> sum
# N₊ = sum(outcome .== statusDict["+"]) # Another variant for N₊ with broadcast operator.
# Number healthy.
N₋ = map(oc -> oc == statusDict["-"], outcome) |> sum
# N₋ = sum(outcome .== statusDict["-"]) # Another variant for N₋ with broadcast operator.

# For each person, randomly determine if the test is positive (+) or negative (-).
accuracy = 0.99
test = fill("", N)
test[outcome .== statusDict["+"]] = sample(signs, Weights([accuracy, 1 - accuracy]), N₊)
test[outcome .== statusDict["-"]] = sample(signs, Weights([1 - accuracy, accuracy]), N₋)

# Present the results.
result = DataFrame(outcome = outcome, test = test)
resultCount = groupby(result, [:outcome, :test]) |>
    grouped -> combine(grouped, nrow => :count)

# Print the confusion matrix.
confusion_matrix(outcome, map(t -> statusDict[t], test))

          ┌───────────────┐
          │ Ground Truth  │
┌─────────┼───────┬───────┤
│Predicted│Disease│Healthy│
├─────────┼───────┼───────┤
│ Disease │  36   │   1   │
├─────────┼───────┼───────┤
│ Healthy │  977  │ 98986 │
└─────────┴───────┴───────┘


## The Hierarchical Model

Some key points to consider:

- Hierarchical models use multiple levels of variability to model results. They are hierarchical because values in the lower levels of the model are computed using values from higher levels of the model.
- We model baseball player batting average using a hierarchical model with two levels of variability:
    - $p \sim N(\mu, \tau)$ describes player-to-player variability in natural ability to hit, which has a mean $\mu$ and standard deviation $\tau$.
    - $Y | p \sim N(p, \sigma)$ describes a player's observed batting average given their ability $p$, which has a mean $p$ and standard deviation $\sigma = \sqrt{p(1 - p)/N}$. This represents variability due to luck.
    - In Bayesian hierarchical models, the first level is called the prior distribution and the second level is called the sampling distribution.
- The posterior distribution allows us to compute the probability distribution of $p$ given that we have observed data $Y$.
- By the continuous version of Bayes' rule, the expected value of the posterior distribution $p$ given $Y = y$ is a weighted average between the prior mean $mu$ and the observed data $Y$:
$$
    \mathbb{E}(p | y) = B\mu + (1 - B) Y \text{ where } B = \frac{\sigma^2}{\sigma^2 + \tau^2}
$$
- The standard error of the posterior distribution $SE(p | Y)^2$ is $\dfrac{1}{\frac{1}{\sigma^2} + \frac{1}{\tau^2}}$. Note that you will need to take the square root of both sides to solve for the standard error.
- This Bayesian approach is also known as shrinking. When $\sigma$ is large, $B$ is close to 1 and our prediction of $p$ shrinks towards the mean ($\mu$). When $\sigma$ is small, $B$ is close to 0 and our prediction of $p$ is more weighted towards the observed data $Y$.